Goal: scrape NIH CSR site for study section members

data source: https://public.csr.nih.gov/StudySections/StandingStudySections

example site: https://public.era.nih.gov/pubroster/preRosIndex.era?AGENDA=438116&CID=102353

https://beautiful-soup-4.readthedocs.io/en/latest/#

In [140]:
# Nested for loop version, keeping original chunks separate below because I still need to figure out how to extract the section ID and date info
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Firefox()

target_url = 'https://public.csr.nih.gov/StudySections/StandingStudySections'

driver.get(target_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

urls = [item.get("href") for item in soup.find_all("a")]
urls_final = [x for x in urls if x.startswith('/StudySections')]

full_urls = []
for ending in urls_final[10:-8]:
    full_urls.append('https://public.csr.nih.gov'+ending)

member_list = list()
for url in full_urls:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    roster_urls = [item.get("href") for item in soup.find_all("a")]
    try:
        roster_url_final = [x for x in roster_urls if x.startswith('https://public.era.nih.gov/pubroster/preRosIndex')]
    except:
        continue
    
    rosters = str(roster_url_final)
    rosters = rosters.split("'")
    rosters = [x for x in rosters if x.startswith('https://public.era.nih.gov/pubroster/preRosIndex')]

    for roster in rosters:
            
        driver.get(roster)

        html = driver.page_source
        soup = BeautifulSoup(html)

        blocks = soup.find_all('p')
        try:
            del blocks[0] # deletes block that doesn't contain member info
        except:
            continue

        for block in blocks:
            block = str(block).replace('\n',"")
            block = block.replace('\t',"")
            block = block.replace('</p>',"")
            block = block.replace('</font>',"")
            block = block.replace('\xa0'," ")
            block = block.replace("    ", "")
            block = block.replace('<p><font color="Navy" size="2"><font color="Navy" size="2">',"")
            block = block.split('<br/>')
            member_list.append(block)

print(member_list)

[['YUAN, CHUN, PHD', 'PROFESSOR', 'DEPARTMENT OF RADIOLOGY AND IMAGING SCIENCES', 'SPENCER FOX ECCLES SCHOOL OF MEDICINE', 'UNIVERSITY OF UTAH', 'SALT LAKE CITY, UT, 84132'], ['APPAVU, BRIAN LEONARD, MD*', 'ASSISTANT PROFESSOR', 'DEPARTMENT OF NEUROLOGY', "PHOENIX CHILDREN'S HOSPITAL", 'PHOENIX, AZ, 85016'], ['BUCKLEY, ERIN MCGUIRE, PHD*', 'ASSISTANT PROFESSOR', 'WALLACE H. COULTER DEPARTMENT OF BIOMEDICAL AT', 'GEORGIA TECH AND EMORY', 'DEPARTMENT OF PEDIATRICS', 'EMORY UNIVERSITY', 'ATLANTA, GA, 30322'], ['BUDDE, MATTHEW D, PHD', 'ASSOCIATE PROFESSOR', 'DEPARTMENT OF NEUROSURGERY', 'MEDICAL COLLEGE OF WISCONSIN', 'MILWAUKEE, WI, 53226'], ['BUSHNELL, CHERYL D, MD', 'PROFESSOR', 'DEPARTMENT OF NEUROLOGY', 'WAKE FOREST SCHOOL OF MEDICINE', 'WINSTON SALEM, NC, 27157'], ['CUEVAS, JAVIER, PHD', 'PROFESSOR', 'DEPARTMENT OF MOLECULAR PHARMACOLOGY AND', 'PHYSIOLOGY', 'COLLEGE OF MEDICINE', 'UNIVERSITY OF SOUTH FLORIDA', 'TAMPA, FL, 33612'], ['DANZER, STEVE C, PHD*', 'PROFESSOR', 'DEPARTMENT O

In [141]:
# Export data to CSV
import pandas as pd # pandas is overkill but I want to learn it dang it
my_df = pd.DataFrame(member_list)
my_df.to_csv('test.csv', index=False, header=False)

In [143]:
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Firefox()

source_url = 'https://public.csr.nih.gov/StudySections/DTCS/MOSS/ACTS'

driver.get(source_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

urls = [item.get("href") for item in soup.find_all("a")]
url_final = [x for x in urls if x.startswith('https://public.era.nih.gov/pubroster/preRosIndex')]

rosters = str(url_final)
rosters = rosters.split("'")
rosters = [x for x in rosters if x.startswith('https://public.era.nih.gov/pubroster/preRosIndex')]

In [144]:
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Firefox()
member_list = list()
for roster in rosters:
        
    driver.get(roster)

    html = driver.page_source
    soup = BeautifulSoup(html)

    blocks = soup.find_all('p')
    del blocks[0] # deletes block that doesn't contain member info

    for block in blocks:
        block = str(block).replace('\n',"")
        block = block.replace('\t',"")
        block = block.replace('</p>',"")
        block = block.replace('</font>',"")
        block = block.replace('\xa0'," ")
        block = block.replace("    ", "")
        block = block.replace('<p><font color="Navy" size="2"><font color="Navy" size="2">',"")
        block = block.split('<br/>')
        member_list.append(block)

print(member_list)

[['TOMIC-CANIC, MARJANA, PHD, RN', 'PROFESSOR OF DERMATOLOGY AND VICE CHAIR OF RESEARCH', 'DEPARTMENT OF DERMATOLOGY AND CUTANEOUS SURGERY', 'DIRECTOR, WOUND HEALING AND REGENERATIVE', 'MEDICINE RESEARCH PROGRAM', 'UNIVERSITY OF MIAMI MILLER SCHOOL OF MEDICINE', 'MIAMI, FL, 33136'], ['ATIT, RADHIKA P, PHD*', 'ASSOCIATE PROFESSOR', 'DEPARTMENT OF BIOLOGY', 'COLLEGE OF ARTS AND SCIENCES', 'CASE WESTERN RESERVE UNIVERSITY', 'CLEVELAND, OH, 44106'], ['BERONJA, SLOBODAN, PHD*', 'ASSOCIATE PROFESSOR', 'HUMAN BIOLOGY DIVISION', 'FRED HUTCHINSON CANCER RESEARCH CENTER', 'SEATTLE, WA, 98109'], ['BOTCHKAREV, VLADIMIR A, MD, PHD*', 'PROFESSOR', 'DEPARTMENT OF DERMATOLOGY', 'SCHOOL OF MEDICINE', 'BOSTON UNIVERSITY', 'BOSTON, MA, 02118'], ['BRASH, ALAN R., PHD*', 'PROFESSOR', 'DEPARTMENT OF PHARMACOLOGY', 'SCHOOL OF MEDICINE', 'VANDERBILT UNIVERSITY', 'NASHVILLE, TN, 37232'], ['CAMPBELL, DANIEL J, PHD', 'MEMBER', 'BENAROYA RESEARCH INSTITUTE', 'VIRGINIA MASON HOSPITAL', 'SEATTLE, WA, 98101'], ['CAP

want to grab and save info on the study section title and date

In [145]:
print(soup.prettify)

<bound method Tag.prettify of <html class="h-100" lang="en"><head>
<!-- Required meta tags -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="IE=edge;" http-equiv="X-UA-Compatible"/>
<link href="/pubroster/images/era/favicon.ico" rel="icon"/>
<!-- CSS -->
<link href="https://public.era.nih.gov/eracdn/public/bootstrap/5.1.3/1.0/css/bootstrap.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://public.era.nih.gov/eracdn/public/font-awesome/5.14.0/css/all.min.css" media="all" rel="stylesheet" type="text/css"/>
<!-- Required JavaScript -->
<!-- jQuery first, then Popper.js, then Bootstrap JS -->
<script src="https://public.era.nih.gov/eracdn/public/jquery/3.5.1/1.0/js/jquery.min.js" type="text/javascript"></script>
<script src="https://public.era.nih.gov/eracdn/public/bootstrap/5.1.3/1.0/js/bootstrap.bundle.min.js" type="text/javascript"></script>
<script defer="" src="/pubroster/js/boots

In [142]:
set = soup.find_all(['center', 'b'])
print(set)

test = [item.get_text for item in soup.find_all(['center', 'b'])]
print(test)



[<center>
<font color="Navy" size="+1">
<b>XENOBIOTIC AND NUTRIENT DISPOSITION AND ACTION STUDY SECTION</b><br/>
</font>
</center>, <b>XENOBIOTIC AND NUTRIENT DISPOSITION AND ACTION STUDY SECTION</b>, <center>
<font color="navy">
<b>XNDA </b>
<br/>
<b>07/08/21 - 07/09/21<br/>
                Meeting Roster</b>
</font>
</center>, <b>XNDA </b>, <b>07/08/21 - 07/09/21<br/>
                Meeting Roster</b>, <b>Notice of NIH Policy to All Applicants:</b>, <b>CHAIRPERSON<br/> ----------------------</b>, <b>MEMBERS<br/> --------------</b>, <b>OTHER REVIEW STAFF<br/> ------------------------------------</b>, <b>SCIENTIFIC REVIEW OFFICER<br/> --------------------------------------------------</b>, <b>EXTRAMURAL SUPPORT ASSISTANT<br/> --------------------------------------------------------</b>, <center>
<i>Last Update:04/08/22
        </i>
</center>]
[<bound method PageElement.get_text of <center>
<font color="Navy" size="+1">
<b>XENOBIOTIC AND NUTRIENT DISPOSITION AND ACTION STUDY SECTION</b